In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 34.9 MB/s 
     |████████████████████████████████| 162 kB 71.5 MB/s 
     |████████████████████████████████| 182 kB 69.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 162 kB 73.2 MB/s 
     |████████████████████████████████| 158 kB 74.0 MB/s 
     |████████████████████████████████| 157 kB 77.7 MB/s 
     |████████████████████████████████| 157 kB 65.8 MB/s 
     |████████████████████████████████| 157 kB 74.7 MB/s 
     |████████████████████████████████| 157 kB 73.8 MB/s 
     |████████████████████████████████| 157 kB 72.0 MB/s 
     |████████████████████████████████| 157 kB 78.9 MB/s 
     |████████████████████████████████| 157 kB 75.5 MB/s 
     |████████████████████████████████| 156 kB 78.2 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wand

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [5]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [4]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbign and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [5]:
resnet50 = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_resnet50=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet50,
  layers.Flatten(),
  layers.Dense(1028, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
resnet50.trainable=False
model_resnet50.summary()

94781440/94765736 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1028)              103162884 
                                                                 
 dense_1 (Dense)             (None, 5)                 5145      
                                                                 
Total params: 126,755,741
Trainable params: 103,168,029
Non-trainable params: 23,587,712
________________________________

In [6]:
model_resnet50.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [7]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "resnet50",
      notes = "without augment and without dropout"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [8]:
%%time
epochs=40
history = model_resnet50.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 24.6814 - accuracy: 0.3851

wandb: Adding directory to artifact (/content/wandb/run-20221012_070938-1ez4f3ip/files/model-best)... Done. 7.9s


19/19 [==============================] - 138s 5s/step - loss: 24.6814 - accuracy: 0.3851 - val_loss: 7.0757 - val_accuracy: 0.4662
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 2.9226 - accuracy: 0.4578

wandb: Adding directory to artifact (/content/wandb/run-20221012_070938-1ez4f3ip/files/model-best)... Done. 8.0s


...

19/19 [==============================] - 44s 2s/step - loss: 0.2513 - accuracy: 0.9206 - val_loss: 0.2461 - val_accuracy: 0.8986
Epoch 33/40
19/19 [==============================] - 5s 209ms/step - loss: 0.2299 - accuracy: 0.9122 - val_loss: 0.5581 - val_accuracy: 0.8041
Epoch 34/40
19/19 [==============================] - 5s 206ms/step - loss: 0.3292 - accuracy: 0.8767 - val_loss: 0.2757 - val_accuracy: 0.9054
Epoch 35/40
19/19 [==============================] - ETA: 0s - loss: 0.2787 - accuracy: 0.8936

wandb: Adding directory to artifact (/content/wandb/run-20221012_070938-1ez4f3ip/files/model-best)... Done. 7.9s


19/19 [==============================] - 42s 2s/step - loss: 0.2787 - accuracy: 0.8936 - val_loss: 0.2406 - val_accuracy: 0.9122
Epoch 36/40
19/19 [==============================] - 4s 178ms/step - loss: 0.2438 - accuracy: 0.9122 - val_loss: 0.4946 - val_accuracy: 0.8176
Epoch 37/40
19/19 [==============================] - ETA: 0s - loss: 0.3136 - accuracy: 0.8936

wandb: Adding directory to artifact (/content/wandb/run-20221012_070938-1ez4f3ip/files/model-best)... Done. 7.8s


19/19 [==============================] - 43s 2s/step - loss: 0.3136 - accuracy: 0.8936 - val_loss: 0.2163 - val_accuracy: 0.9122
Epoch 38/40
19/19 [==============================] - 5s 214ms/step - loss: 0.2251 - accuracy: 0.9274 - val_loss: 0.3003 - val_accuracy: 0.8851
Epoch 39/40
19/19 [==============================] - 5s 204ms/step - loss: 0.2611 - accuracy: 0.9037 - val_loss: 0.2627 - val_accuracy: 0.8851
Epoch 40/40
19/19 [==============================] - 4s 181ms/step - loss: 0.3129 - accuracy: 0.8801 - val_loss: 0.6455 - val_accuracy: 0.7162
CPU times: user 6min 59s, sys: 1min 56s, total: 8min 55s
Wall time: 17min


In [9]:
wandb.finish()

accuracy,▁▂▄▆▅▅▆▆▇▆▇█▇▆▆▆▇█▇▇▇▇▇██▇███▇▇██▇▇█▇██▇
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▅▃▄▅▅▇▆▅▇▇▆▆▂▇█▇▆▇▇▇▆██▇▇▇▆▄▇█▆██▇███▅
val_loss,█▃▁▂▂▁▂▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
accuracy,0.88007
best_epoch,36
best_val_loss,0.21628
epoch,39
loss,0.3129
val_accuracy,0.71622
